This notebook is meant for our lecture 4 breakout session.  I've put together some questions to guide your [here](https://itempool.com/MIT-Robotic-Manipulation/c/yzVbTp9zFc_).  The points are not real, and will not contribute to your grade!

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [2]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  version='20200909'
  build='nightly'
  urlretrieve(f"https://drake-packages.csail.mit.edu/drake/{build}/drake-{version}/setup_drake_colab.py",
              "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version=version, build=build)

# Install pyngrok.
server_args = []
if 'google.colab' in sys.modules:
  !pip install pyngrok
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Let's do all of our imports here, too.
import numpy as np
import altair as alt
import pydot
from IPython.display import display, SVG
from ipywidgets import Text, Textarea, Layout

from pydrake.all import (
    AddMultibodyPlantSceneGraph, AngleAxis, BasicVector, ConnectMeshcatVisualizer, 
    DiagramBuilder, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
    LeafSystem, MultibodyPlant, MultibodyPositionToGeometryPose, Parser, 
    PiecewisePolynomial, PiecewiseQuaternionSlerp, Quaternion, RigidTransform, 
    RollPitchYaw, RotationMatrix, SceneGraph, Simulator, TrajectorySource,
    DifferentialInverseKinematicsParameters, DifferentialInverseKinematicsIntegrator,
    set_log_level
)
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from pydrake.examples.manipulation_station import ManipulationStation

from pydrake.all import (DiagramBuilder, AddMultibodyPlantSceneGraph,
                         ConnectPlanarSceneGraphVisualizer, Parser,
                         PiecewisePolynomial)
from manipulation.utils import FindResource


# TODO(russt): upstream this to drake
import numpy as np
from functools import partial

from IPython.display import display
from ipywidgets import FloatSlider, Layout, ToggleButton, ToggleButtons

from pydrake.all import JointIndex

def MakeJointSlidersThatPublishOnCallback(
    plant, publishing_system, root_context, my_callback=None, 
    lower_limit=-10., upper_limit=10., resolution=0.01, length=200):
    """ 
    Creates an ipywidget slider for each joint in the plant.  Unlike the 
    JointSliders System, we do not expect this to be used in a Simulator.  It 
    simply updates the context and calls Publish directly from the slider 
    callback.
    
    Args:
        plant:        A MultibodyPlant.
        publishing_system: The System whos Publish method will be called.  Can 
                           be the entire Diagram, but can also be a subsystem.
        root_context: A mutable root Context of the Diagram containing both the 
                      ``plant`` and the ``publishing_system``; we will extract 
                      the subcontext's using `GetMyContextFromRoot`.
        my_callback:  An optional additional callback function that will be 
                      called when the widgets update, using  
                      ``my_callback(plant_context)``.
        lower_limit:  A scalar or vector of length robot.num_positions().
                      The lower limit of the slider will be the maximum
                      value of this number and any limit specified in the
                      Joint.
        upper_limit:  A scalar or vector of length robot.num_positions().
                      The upper limit of the slider will be the minimum
                      value of this number and any limit specified in the
                      Joint.
        resolution:   A scalar or vector of length robot.num_positions()
                      that specifies the discretization of the slider.
        length:       The length of the sliders.
    
    Returns:
        A list of the slider widget objects that are created.

    Note: Some publishers (like MeshcatVisualizer) use an initialization event 
    to "load" the geometry.  You should call that *before* calling this method 
    (e.g. with `meshcat.load()`).
    """

    def _reshape(x, num):
        x = np.array(x)
        assert len(x.shape) <= 1
        return np.array(x) * np.ones(num)

    lower_limit = _reshape(lower_limit, plant.num_positions())
    upper_limit = _reshape(upper_limit, plant.num_positions())
    resolution = _reshape(resolution, plant.num_positions())

    publishing_system_context = publishing_system.GetMyContextFromRoot(root_context)
    plant_context = plant.GetMyContextFromRoot(root_context)
    positions = plant.GetPositions(plant_context)

    # Publish once immediately.
    publishing_system.Publish(publishing_system_context)
    if my_callback:
        my_callback(plant_context)

    def _slider_callback(change, index):
        positions[index] = change.new
        plant.SetPositions(plant_context, positions)
        publishing_system.Publish(publishing_system_context)
        if my_callback:
            my_callback(plant_context)

    slider_widgets = []
    k = 0
    for i in range(0, plant.num_joints()):
        joint = plant.get_joint(JointIndex(i))
        low = joint.position_lower_limits()
        upp = joint.position_upper_limits()
        for j in range(0, joint.num_positions()):
            index = joint.position_start() + j
            slider = FloatSlider(
                value=positions[index],
                min=max(low[j], lower_limit[k]),
                max=min(upp[j], upper_limit[k]),
                step=resolution[k],
                continuous_update=True,
                description=joint.name(),
                style={'description_width': 'initial'},
                layout=Layout(width=f"'{length}'"))
            slider.observe(partial(_slider_callback, index=index), names='value')
            display(slider)
            slider_widgets.append(slider)
            k += 1

    return slider_widgets


import meshcat.geometry as g
import meshcat.transformations as tf

def plot_surface(meshcat_viz, X, Y, Z, color=0xdd9999):
    (rows, cols) = Z.shape

    vertices = np.empty((rows*cols,3),dtype=np.float32)
    vertices[:,0] = X.reshape((-1))
    vertices[:,1] = Y.reshape((-1))
    vertices[:,2] = Z.reshape((-1))

    # Vectorized faces code from https://stackoverflow.com/questions/44934631/making-grid-triangular-mesh-quickly-with-numpy
    faces = np.empty((cols-1,rows-1,2,3),dtype=np.uint32)
    r = np.arange(rows*cols).reshape(cols,rows)
    faces[:,:, 0,0] = r[:-1,:-1]
    faces[:,:, 1,0] = r[:-1,1:]
    faces[:,:, 0,1] = r[:-1,1:]
    faces[:,:, 1,1] = r[1:,1:]
    faces[:,:, :,2] = r[1:,:-1,None]
    faces.shape =(-1,3)
    meshcat_viz.set_object(
        g.TriangularMeshGeometry(vertices, faces), 
        g.MeshLambertMaterial(color=color, wireframe=False))

# Differential Inverse Kinematics as a Quadratic Program


In [11]:
import time
def BuildAndVisualize():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    twolink = Parser(plant, scene_graph).AddModelFromFile(
        FindResource("models/double_pendulum.urdf"))
    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(builder,
                                  scene_graph,
                                  zmq_url=zmq_url)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    T = 2.
    q = PiecewisePolynomial.FirstOrderHold(
        [0, T, 2*T], np.array([[-np.pi / 2.0 + 1., -np.pi / 2.0 - 1., -np.pi / 2.0 + 1.], 
                               [-2., 2., -2]]))
    plant_context = plant.GetMyContextFromRoot(context)

    meshcat.load()
    for i in range(2):
        for t in np.linspace(0, 2*T, num=100):
            context.SetTime(t)
            plant.SetPositions(plant_context, q.value(t))
    #        meshcat.vis["QP"]
            diagram.Publish(context)
            time.sleep(0.05)

BuildAndVisualize()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.
